In [4]:
import os
from matplotlib import pyplot as plt
import rasterio
import numpy as np
from skimage import exposure

def contrast_stretch(band, band_small, band_large):
    band_small, band_large = np.percentile(band, (percent_small, percent_large))
    scaled = exposure.rescale_intensity(band, in_range=(band_small, band_large), out_range=np.float32)
    return scaled

def normalize_img(img):
    return (img-np.min(img))/(np.max(img)-np.min(img))

percent_small = 0.2 ; percent_large = 100 - percent_small ; 

path = "/mnt/hdd/eric/.tmp_ipy/00.Data/K3_20221211/_262"
dir_list = os.listdir(path)

red   = os.path.join( path, [i for i in os.listdir(path) if i.split(".")[0][-1]=="R" ][0] )
green = os.path.join( path, [i for i in os.listdir(path) if i.split(".")[0][-1]=="G" ][0] )
blue  = os.path.join( path, [i for i in os.listdir(path) if i.split(".")[0][-1]=="B" ][0] )

dat_r = rasterio.open(red, dtype = 'float64')
dat_g = rasterio.open(green, dtype = 'float64')
dat_b = rasterio.open(blue, dtype = 'float64')

In [10]:
dat_r.read().shape

dat_r_reshape = np.reshape(dat_r.read(), dat_r.shape)
print("r shape : ",dat_r.shape)

r shape :  (26880, 32912)


In [11]:
dat_g.shape
dat_g_reshape = np.reshape(dat_g.read(), dat_g.shape)
print("g shape : ",dat_g.shape)

g shape :  (26880, 32912)


In [12]:
dat_b.shape
dat_b_reshape = np.reshape(dat_b.read(), dat_b.shape)
print("b shape : ",dat_b.shape)

ValueError: cannot reshape array of size 2654023680 into shape (26880,32912)

In [ ]:
dat_r_reshape = np.reshape(dat_r.read(), dat_r.shape)
print("r shape : ",dat_r.shape)
dat_g_reshape = np.reshape(dat_g.read(), dat_g.shape)
print("g shape : ",dat_g.shape)
dat_b_reshape = np.reshape(dat_b.read(), dat_b.shape)
print("b shape : ",dat_b.shape)


In [ ]:

contrast_flag = True
if contrast_flag ==True:
    r_con = contrast_stretch(dat_r_reshape, 1, 99)
    g_con = contrast_stretch(dat_g_reshape, 1, 99)
    b_con = contrast_stretch(dat_b_reshape, percent_small, percent_large)
    
    print(r_con.shape,g_con.shape,b_con.shape)

    img_temp = np.stack((r_con, g_con, b_con), axis=0)

else:
    img_temp = np.stack((dat_r_reshape,dat_g_reshape,dat_b_reshape), aixs=0)

RGB = normalize_img(img_temp) # save for tiff

# plt.imsave(os.path.join(path,"_RGB_v1.png"),RGB )

In [36]:
RGB.shape

(3, 26844, 32724)

In [37]:
# Make tiff file
outpath = os.path.join( path, "_RGB.tif")

# Open multiband raster
raster = rasterio.open(red)

# Write to TIFF
kwargs = raster.meta
kwargs.update(
    dtype=rasterio.float32,
    count=3,
    compress='lzw')

with rasterio.open(outpath, 'w', **kwargs) as dst:
    dst.write(RGB.astype(rasterio.float32))
raster.close()
dst.close()
